# Finding the optimal time of the day to post on Haker News

[Hacker News](https://news.ycombinator.com) is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

The dataset can be found [here](https://www.kaggle.com/hacker-news/hacker-news-posts/home), but note that it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.

As our first steps we:
- Read the hacker_news.csv file in a list of lists.
- Display the first five rows of the file.

In [1]:
from csv import reader

open_file = open('hacker_news.csv')
read_file = reader(open_file)
hn = list(read_file)
print(hn[:5])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


In the next cell we:
- Extract the first row of data, and assign it to the variable headers.
- Remove the first row from hn.
- Display headers.
- Display the first five rows of hn to verify that we removed the header row properly.

In [2]:
headers = hn[0]
hn = hn[1:]
print(headers)
print("-----------------------------------------------------------")
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
-----------------------------------------------------------
[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0',

We're specifically interested in posts whose titles begin with either **Ask HN** or **Show HN**. Users submit **Ask HN** posts to ask the Hacker News community a specific question. Likewise, users submit **Show HN** posts to show the Hacker News community a project, product, or just generally something interesting.

Next step: filter to separate lists posts starting with 'ask hn', 'show hn' and others.

In [3]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)

print('Posts starting with "ask hn": ', len(ask_posts))
print('Posts starting with "show hn": ',len(show_posts))
print('Other posts: ',len(other_posts))

Posts starting with "ask hn":  1744
Posts starting with "show hn":  1162
Other posts:  17194


Next, let's determine if ask posts or show posts receive more comments on average.

In [4]:
total_ask_comments = 0

for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments
    
avg_ask_comments = total_ask_comments / len(ask_posts)
print('Average number of comments for ask posts: ', avg_ask_comments)

total_show_comments = 0

for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments
    
avg_show_comments = total_show_comments / len(show_posts)
print('Average number of comments for show posts: ', avg_show_comments)

Average number of comments for ask posts:  14.038417431192661
Average number of comments for show posts:  10.31669535283993


**Ask posts** have more comments then **Show posts**. Which makes sense, because users more likely to answer questions asked rather then just comment where there is no question.

Since **Ask posts** are more likely to receive comments, we'll focus our remaining analysis just on these posts.

Next, we'll **determine if ask posts created at a certain time are more likely to attract comments**. We'll use the following steps to perform this analysis:

1. Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments ask posts receive by hour created.

In [5]:
import datetime as dt

result_list = []

for row in ask_posts:
    result_list.append([row[6], int(row[4])])

counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    created_at = dt.datetime.strptime(row[0], "%m/%d/%Y %H:%M")
    hour = created_at.strftime("%H")
    if hour in counts_by_hour:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += row[1]
    else:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = row[1]
        

Next, we'll use these two dictionaries to calculate the average number of comments for posts created during each hour of the day.

In [6]:
avg_by_hour = []

for hour in counts_by_hour:
   avg_by_hour.append([hour, comments_by_hour[hour]/counts_by_hour[hour]])
    
print(avg_by_hour)

[['03', 7.796296296296297], ['23', 7.985294117647059], ['19', 10.8], ['02', 23.810344827586206], ['11', 11.051724137931034], ['04', 7.170212765957447], ['18', 13.20183486238532], ['09', 5.5777777777777775], ['05', 10.08695652173913], ['16', 16.796296296296298], ['14', 13.233644859813085], ['22', 6.746478873239437], ['21', 16.009174311926607], ['01', 11.383333333333333], ['08', 10.25], ['12', 9.41095890410959], ['00', 8.127272727272727], ['10', 13.440677966101696], ['15', 38.5948275862069], ['13', 14.741176470588234], ['20', 21.525], ['07', 7.852941176470588], ['06', 9.022727272727273], ['17', 11.46]]


Although we now have the results we need, this format makes it hard to identify the hours with the highest values. Let's finish by sorting the list of lists and printing the five highest values in a format that's easier to read.

In [7]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
    
print(swap_avg_by_hour)
sorted_swap = sorted(swap_avg_by_hour, reverse=True)
print("\nTop 5 Hours for Ask Posts Comments:\n")
for row in sorted_swap[:5]:
    print("{0}:00: {1:.2f}".format(row[1], row[0]))

[[7.796296296296297, '03'], [7.985294117647059, '23'], [10.8, '19'], [23.810344827586206, '02'], [11.051724137931034, '11'], [7.170212765957447, '04'], [13.20183486238532, '18'], [5.5777777777777775, '09'], [10.08695652173913, '05'], [16.796296296296298, '16'], [13.233644859813085, '14'], [6.746478873239437, '22'], [16.009174311926607, '21'], [11.383333333333333, '01'], [10.25, '08'], [9.41095890410959, '12'], [8.127272727272727, '00'], [13.440677966101696, '10'], [38.5948275862069, '15'], [14.741176470588234, '13'], [21.525, '20'], [7.852941176470588, '07'], [9.022727272727273, '06'], [11.46, '17']]

Top 5 Hours for Ask Posts Comments:

15:00: 38.59
02:00: 23.81
20:00: 21.52
16:00: 16.80
21:00: 16.01


#### Conclusion:
To have a higher chance of receiving comments user should create a post at 15 (3 a.m.). This is for Eastern Time zone in the US, so in Pacific Standard Time zone it should be 12 (12 noon).